- 궁극적인 목표는 보다 견고하고 결함이 없는 코드를 만드는 것
- 좀 더 높은 수준의 추상화를 할 수 있도록 도와주는 디자인 원칙을 알아보자
- 이번 장의 목표
    - 견고한 소프트웨어의 개념을 이해
    - 작업 중 잘못된 데이터를 다루는 방법
    - 새로운 요구 사항을 쉽게 받아들이고 확장할 수 있는 유지보수가 쉬운 소프트웨어 설계
    - 재사용 가능한 소프트웨어 설계
    - 개발팀의 생산성을 높이는 효율적인 코드 작성

# 계약에 의한 디자인
- 캡슐화
    - 컴포넌트는 기능을 숨기고 클라이언트에게 API를 노출해야 한다
    - 함수, 클래스, 메서드는 특별한 유의사항에 따라 동작해야 하며, 그렇지 않을 경우 코드가 깨지게 된다.
- 클라이언트는 기대에 따라 행동
    - 클라이언트가 잘못 파라미터를 호출하면 실행이 되면 안된다
    - 조용히 오류를 지나치면 안된다
- 계약
    - API를 디자인할때 예상되는 입출력과 부작용을 문서화해야 하지만, 런타임 시의 소프트웨어 동작까지 강제할 수는 없다.
    - 코드가 정상적으로 동작하기 위해 필요한 것과 클라이언트가 반환받게 될 형태는 모두 디자인에 포함이 되어야 한다
- 계약에 의한 디자인
    - 양측이 동의하는 계약을 먼저 한다
    - 계약을 어긴 경우 명시적으로 예외를 발생시킨다
    - 오류를 쉽게 찾아낼 수 있고, 잘못된 가정 하에 코드의 핵심 부분이 실행되는 것을 방지할 수 있다. 
    - 사전 조건 (precondition)
        - 코드가 실행되기 전에 체크해야 하는 것들
        - 함수가 진행되기 전에 조건을 체크
        - 데이터의 유효성 검사를 많이 하자 (데이터베이스, 파일, 이전에 호출된 다른 메서드도 검사!)
    - 사후 조건 (postcondition)
        - 함수 반환값의 유효성 검사
        - 컴포넌트가 기대한 것을 제대로 받았는지 확인하기 위해 수행
    - 불변식 (invariant)
        - 함수가 실행되는 동안에 일정하게 유지되는 것으로 함수의 로직에 문제가 없는지 확인하기 위한 것
    - 부작용 (side-effect)
        - 선택적으로 부작용을 docstring에 언급할 수 있다

In [6]:
def test(a: int, b: int) -> int:
    return a + b

In [7]:
test(1, 2)

3

In [8]:
test(1, '2')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [9]:
def test(a: int, b: int) -> int:
    if not isinstance(a, int) or not isinstance(b, int):
        raise TypeError('a and b must be integers')

    return a + b

In [10]:
test(1, '2')

TypeError: a and b must be integers

## 사전 조건 (precondition)
- Input 데이터가 적절한 형태여야 함
- 단순한 타입 체크가 아니라 필요로 하는 값이 정확한지 확인하는 것에 가까움
- 까다로운 접근 방법: 함수가 자체적으로 로직을 실행하기 전에 유효성 검사를 한다.
    - 일반적으로 가장 안전하고 견고함, 업계에서도 널리 쓰임
- 중복 제거 원칙: 함수 밖에서 검증하든 안에서 검증하든 어느 한쪽에서만 해야 한다

## 사후 조건 (postcondition)
- 메서드 또는 함수가 반환된 후의 상태를 강제하는 것
- 반환 객체를 문제 없이 사용할 수 있는가?를 검증하는 것

## 파이썬스러운 계약
- PEP-316: Programming by Contract for Python
    - 연기된 deferred 상태이지만... 연기되었다고 하여 이 원칙을 파이썬으로 구현할 수 없다는 뜻은 아니다!
    - 메서드, 함수, 클래스에 제어 메커니즘을 추가하고 검사에 실패할 경우 RuntimeError나 ValueError를 발생시키자
        - 사용자 정의 예외를 만드는 것도 좋다
    - 코드를 가능한한 격리된 상태로 유지하자
        - 사전 조건 / 사후 조건 / 핵심 기능 구현을 구분하자
        - 데코레이터를 사용하는 것도 좋은 방법

## 계약에 의한 디자인 (DbC) - 결론
- 문제가 있는 부분을 효과적으로 식별하는 데 가치가 있다
- 계약을 정의 -> 런타임 오류가 발생했을 때 무엇이 계약을 파손시켰는지 명확해짐 -> 코드가 견고해짐
- 코드의 구조가 명확해짐
- 추가 작업이 발생하지만 장기적인 보상을 받을 수 있다
- 무엇을 검증할 것인지 신중한 검토가 필요하다
    - 단순한 데이터 타입만 검사하는 것은 별로 의미가 없다

# 방어적 defensive 프로그래밍
- 계약에 의한 디자인과는 다소 다른 접근 방식
    - 객체, 함수, 메서드의 모든 부분을 유효하지 않은 것으로부터 스스로 보호할 수 있게 하는 것
    - 다른 철학을 가졌다기 보다는 서로 보완 관계에 있을 수 있다는 것을 의미
- 주요 주제
    - 예상할 수 있는 시나리오의 오류를 처리하는 방법: 에러 핸들링 프로시저
    - 발생하지 않아야 하는 오류를 처리하는 방법: assertion


## 에러핸들링
- 에러에 대해서 실행을 계속할 수 있을지 프로그램을 중단할지 결정하는 것
- 에러 처리 방법
    - 값 대체 (value substitution)
    - 에러 로깅
    - 예외 처리

### 값 대체
- 소프트웨어가 잘못된 값을 생성하거자 전체가 종료될 위험이 있을 경우 안전한 다른 값으로 대체할 수 있다
- 기본 값, 잘 알려진 상수, 초기 값으로...
- 견고성과 정확성 간의 트레이드오프
    - 대체 값이 실제로 안전한 옵션인 경우에 한해 신중하게 선택해야 한다.
    - 소프트웨어는 예상치 못한 상황에서도 실패하지 않아야 견고하다.
    - 하지만 무조건 실패하지 않는 것이 항상 옳은 것은 아니다.
- 민감하고 중요한 정보를 다루는 소프트웨어의 경우 부정확한 결과를 내보내는 것 보다는 정확하더라도 대체 값을 사용하지 않는 것이 낫다.
- 안전을 위하여 기본 값을 제공할 수도 있다.
    - 설정되지 않은 환경변수의 기본 값
    - 설정 파일의 누락된 항목
    - 함수의 파라미터의 기본 값

In [4]:
configuration = {"dbport": 5432}
configuration.get("dbhost", "localhost")

'localhost'

In [2]:
configuration.get("dbport")

5432

In [5]:
import os

os.getenv("DBHOST")

In [11]:
os.getenv("DBPORT", 5432)

5432

### 예외 처리
- 잘못된 값을 입력한 경우에 복구하기 보다는 차라리 실행을 멈추고 호출자에게 빨리 알리는 것이 좋을 수 있다
- 외부 컴포넌트 중 하나의 문제로 인한 예외가 발생할 수 있다
    - 적절하게 인터페이스를 설계하면 쉽게 디버깅할 수 있다
- 예외 메커니즘: 예외적인 상황을 명확하게 알려주고 원래의 비즈니스 로직에 따라 흐름을 유지하자
- 그치만 예외를 go-to 문처럼 쓰면 안된다.... 호출자가 알아야만 하는 실질적인 문제가 있을 경우에만 예외를 발생시키자

- 예외는 캡슐화를 약화시킨다
    - 호출자에게 잘못을 알려주는 것 -> 호출자가 함수에 대해 더 많은 것을 알아야만 한다는 것
    - 너무 많은 예외를 발생시켜버리면 문맥에서 자유롭지 않다는 것을 의미
        - 왜냐하면 호출할때마다 발생 가능한 부작용을 염두에 둬야 하기 때문!
        - 이런 경우 차라리 여러 기능으로 쪼갤 수 있는 지 검토해 보자


#### 올바른 수준의 추상화 단계에서 예외 처리
- 함수 - 오직 한 가지 일만 해야 함
- 예외 - 그러한 함수의 한 부분이어야 한다
- 함수가 처리하는 / 발생시키는 예외는 함수가 캡슐화하고 있는 "로직"에 대한 것이어야 한다
- 서로 다른 추상화를 혼합하는 예제

In [13]:
import logging
import time


logger = logging.getLogger(__name__)


class Connector:
    """Abstract the connection to a database."""

    def connect(self):
        """Connect to a data source."""
        return self

    @staticmethod
    def send(data):
        return data


class Event:
    def __init__(self, payload):
        self._payload = payload

    def decode(self):
        return f"decoded {self._payload}"


def connect_with_retry(
    connector: Connector, retry_n_times: int, retry_backoff: int = 5
):
    """Tries to establish the connection of <connector> retrying
    <retry_n_times>, and waiting <retry_backoff> seconds between attempts.

    If it can connect, returns the connection object.
    If it's not possible to connect after the retries have been exhausted, raises ``ConnectionError``.

    :param connector:         An object with a ``.connect()`` method.
    :param retry_n_times int: The number of times to try to call
                                ``connector.connect()``.
    :param retry_backoff int: The time lapse between retry calls.

    """
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: attempting new connection in %is", e, retry_backoff
            )
            time.sleep(retry_backoff)
    exc = ConnectionError(f"Couldn't connect after {retry_n_times} times")
    logger.exception(exc)
    raise exc


class DataTransport:
    """An example of an object handling exceptions of different levels."""

    _RETRY_BACKOFF: int = 5
    _RETRY_TIMES: int = 3

    def __init__(self, connector: Connector) -> None:
        self._connector = connector
        self.connection = None

    def deliver_event(self, event: Event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection error detected: %s", e)
            raise
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

    def connect(self):
        for _ in range(self._RETRY_TIMES):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: attempting new connection in %is",
                    e,
                    self._RETRY_BACKOFF,
                )
                time.sleep(self._RETRY_BACKOFF)
            else:
                return self.connection
        raise ConnectionError(
            f"Couldn't connect after {self._RETRY_TIMES} times"
        )

    def send(self, data: bytes):
        return self.connection.send(data)

- `deliver_event` 메서드의 ValueError 와 ConnectionError는 관계가 없다
    - ConnectionError는 connect 메서드 내에서 처리되어야 한다
    - 그러므로 이 두 부분을 분리하여야 한다

In [ ]:
def connect_with_retry(
    connector: Connector, retry_n_times: int, retry_backoff: int = 5
):
    """Tries to establish the connection of <connector> retrying
    <retry_n_times>, and waiting <retry_backoff> seconds between attempts.

    If it can connect, returns the connection object.
    If it's not possible to connect after the retries have been exhausted, raises ``ConnectionError``.

    :param connector:         An object with a ``.connect()`` method.
    :param retry_n_times int: The number of times to try to call
                                ``connector.connect()``.
    :param retry_backoff int: The time lapse between retry calls.

    """
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: attempting new connection in %is", e, retry_backoff
            )
            time.sleep(retry_backoff)
    exc = ConnectionError(f"Couldn't connect after {retry_n_times} times")
    logger.exception(exc)
    raise exc


class DataTransport:
    """An example of an object that separates the exception handling by
    abstraction levels.
    """

    _RETRY_BACKOFF: int = 5
    _RETRY_TIMES: int = 3

    def __init__(self, connector: Connector) -> None:
        self._connector = connector
        self.connection = None

    def deliver_event(self, event: Event):
        self.connection = connect_with_retry(
            self._connector, self._RETRY_TIMES, self._RETRY_BACKOFF
        )
        self.send(event)

    def send(self, event: Event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

#### 엔드 유저에게 Traceback 노출 금지
- 보안을 위한 고려 사항
    - 예외 처리 시 오류 발생 사실이 너무 중요하다면 전파하는 것이 가능하지만,
    - 검토된 특정 시나리오이거나 견고함보다 정확성이 중요한 경우 등에는 프로그램을 바로 중단할수도 있다.
- traceback 정보나 메시지, 기타 수집 가능한 정보를 최대한 로그로 남기되 절대 사용자에게 보여서는 안된다.
    - 중요 정보나 지적 재산이 유출될 위험이 있다
- "알 수 없는 문제가 발생했습니다" 또는 "페이지를 찾을 수 없습니다"

#### 비어있는 except 블록 코드 지양
- 안티 패턴 중에서도 가장 악마같은 패턴 (REAL 01?)
    - 아무것도 하지 않은 채로 조용히 지나쳐버리는 비어있는 except 블록
- 에러는 결코 조용히 전달되어서는 안된다 - The Zen of Python

In [17]:
def process_data():
    # do something
    pass

try:
    process_data()
except:
    pass

- 대안
    - 구체적인 예외를 사용하자 (Exception 같은걸 쓰지 말자)
        - AttributeError or KeyError
    - except 블록에서 실제 오류 처리를 하자
        - 로깅을 하거나
        - 기본 값을 반환하거나 (오류를 발견한 뒤에만 사용하는 값)
        - 기본 오류와 다른 새로운 오류를 발생시키거나
            - 이 경우 원본 예외를 함께 전달하자
    - 둘다 하는 것이 좋다
- pass가 나쁜 이유
    - 의미하는 바를 알 수 없다
        - 실제로 해당 오류를 무시하기 위해서인지 아닌지...
        - 오류를 무시하려면 `contextlib.suppress`를 사용합시다

In [18]:
import contextlib

with contextlib.suppress(KeyError):
    process_data()

#### 원본 예외 포함
- 새 오류 메시지를 발생시킬 때 원본 메시지를 포함시키는 방법
- PEP-3134: Exception Chaining and Embedded Tracebacks
    - `raise e from original_exception` 
    - `__cause__` 속성에 할당된다.

In [19]:
class InternalDataError(Exception):
    """Exception for representing an error in the data container."""
    def process(data_dictionary, record_id):
        try:
            return data_dictionary[record_id]
        except KeyError as e:
            raise InternalDataError(
                f"Record {record_id} is not present in the data set"
            ) from e

### 파이썬에서 assertion 사용하기
- 절대 일어나지 않아야 하는 상황에 사용되는 것, 불가능한 조건
- 소프트웨어에 결함이 있음을 의미
- assert 문을 비즈니스 로직과 섞거나 소프트웨어 제어 흐름 메커니즘으로 사용하지는 말자

In [ ]:
# Bad example
try:
    assert condition.holds(), "Condition not met"
except AssertionError:
    alternative_procedure()

- Assertion이 뜨면 프로그램이 종료되어야 한다
- 파이썬 쓸 때 -O 플래그와 함께 실행하면 assert 문을 무시하도록 할 수 있다
    - 하지만 장려하지 않음
- Assertion 문이 함수이면 안된다. 
- 예외와 assert의 차이
    - 예외는 예상치 못한 상황을 처리하기 위한 것
    - assert는 정확성을 보장하기 위해 스스로 체크하기 위한 것
    - 예외가 일반적

# 관심사의 분리
- 책임이 다르면 컴포넌트, 계층 또는 모듈로 분리되어야 한다
- 파급 ripple 효과
    - 어느 지점에서의 변화가 전체로 전파되는 것
    - 함수 정의를 약간만 변경해도 코드 여러 부분에 영향을 미쳐 많은 코드를 변경해야 하는 것

## 응집력 cohesion 과 결합력 coupling
- 응집력
    - 객체가 작고 잘 정의된 목적을 가져야 함
- 결합력
    - 두 개 이상의 객체가 어떻게 의존하는지
    - 너무 의존적이라면 바람직하지 않음
        - 낮은 재사용성: 지나치게 의존하거나 너무 많은 파라미터를 가진 경우 결합력이 높음 -> 다른 상황에서는 사용하기 어려움 
        - 파급 효과: 두 부분 중 하나를 변경하면 다른 부분에도 영향을 미침
        - 낮은 수준의 추상화: 서로 다른 추상화 레벨에서 문제를 해결하기 어려워 관심사가 분리되어 있다고 보기 어려움
- 높은 응집력, 낮은 결합력!

# 개발 지침 약어
## DRY / OAOO
- DRY: Do not Repeat Yourself
- OAOO: Once And Only Once
- 설명
    - 코드에 있는 지식은 단 한번 단 한 곳에 정의되어야 한다.
    - 코드 중복은 유지보수에 직접적인 영향을 미친다
- 단점
    - 오류가 발생하기 쉽다: 수정했을 때 하나라도 빠뜨리면 버그가 발생
    - 비용이 비싸다: 변경하는데 더 많은 시간이 소요된다
    - 신뢰성이 떨어진다: 데이터의 완결성이 떨어진다

In [ ]:
def process_students_list(students):
    students_ranking = sorted(
        students, key=lambda s: s.passed * 11 - s.failed * 5 - s.years * 2
    )

    for student in students_ranking:
        print(
            "이름: {0}, 점수: {1}".foramt(
                student.name, 
                student.passed * 11 - student.failed * 5 - student.years * 2
            )
        )

## YAGNI
- You Ain't Gonna Need It
    - 과잉 엔지니어링을 하지 않기 위해 염두에 두어야 하는 원칙
- 유지 보수가 가능한 소프트웨어를 만들기 != 미래의 요구 사항을 예측하기
    - 현재의 요구사항을 잘 해결하기 위한 소프트웨어를 작성하고 가능한 나중에 수정하기 쉽도록 작성하자
- 상세 코드 수준 / 전체적인 소프트웨어 아키텍쳐 수준에서 적용할 수 있음

## KIS
- Keep It Simple
    - 소프트웨어 컴포넌트를 설계할 때 과잉 엔지니어링을 피해야 한다
    - 디자인이 단순할수록 유지 관리가 쉽다
- 역시 모든 추상화 수준에서 염두에 두어야 한다

In [21]:
class ComplicatedNamespace:
    """An convoluted example of initializing an object with some properties.

    >>> cn = ComplicatedNamespace.init_with_data(
    ...    id_=42, user="root", location="127.0.0.1", extra="excluded"
    ... )
    >>> cn.id_, cn.user, cn.location
    (42, 'root', '127.0.0.1')

    >>> hasattr(cn, "extra")
    False

    """

    ACCEPTED_VALUES = ("id_", "user", "location")

    @classmethod
    def init_with_data(cls, **data):
        instance = cls()
        for key, value in data.items():
            if key in cls.ACCEPTED_VALUES:
                setattr(instance, key, value)
        return instance


class Namespace:
    """Create an object from keyword arguments.

    >>> cn = Namespace(
    ...    id_=42, user="root", location="127.0.0.1", extra="excluded"
    ... )
    >>> cn.id_, cn.user, cn.location
    (42, 'root', '127.0.0.1')

    >>> hasattr(cn, "extra")
    False
    """

    ACCEPTED_VALUES = ("id_", "user", "location")

    def __init__(self, **data):
        for attr_name, attr_value in data.items():
            if attr_name in self.ACCEPTED_VALUES:
                setattr(self, attr_name, attr_value)

## EAFP/LBYL
- Easier to Ask Forgiveness than Permission
    - 먼저 코드를 실행하고 실제 동작하지 않을 경우에 대응한다
    - 일단 코드를 실행하고 발생한 예외를 catch하고 except 블록에서 바로잡는 코드를 실행
- Look Before You Leap
    - 먼저 무엇을 사용하려고 하는지 확인한 후 실행

In [ ]:
# EAFP
try:
    with open(filename) as f:
        content = f.read()
except FileNotFoundError:
    logger.error("File not found")

# LBYL
if os.path.exists(filename):
    with open(filename) as f:
        content = f.read()

- 대부분 EAFP, 특수한 경우 LBYL
    - EAFP가 의미가 명확하고 가독성이 높다

# 상속
- 상속은 결합력이 높은 클래스 관계를 만든다
- 코드 재사용 - 단지 부모 클래스의 메서드를 얻기 위해 상속하는 것은 좋지 않은 생각
- 코드를 재사용하고 싶다? 조합해서 써라


## 상속이 좋은 선택인 경우
- 파생 클래스는 양날의 검
    - 부모 클래스를 공짜로 전수받지만 너무 많은 기능을 불필요하게 추가하게 되기도
- 상속된 모든 메서드를 실제로 사용할 것인가?
- 설계상의 실수인 것
    - 상위 클래스는 잘 정의된 인터페이스 (x) 막연한 정의와 너무 많은 책임
    - 하위 클래스는 확장하려고 하는 상위 클래스의 적절한 세분화가 아님
- 이 클래스와 같은 기능을 하지만 일부 기능을 수정하거나 새로운 것을 추가하고 싶어서 상속을 한 경우 Good


## 상속 안티 패턴
- 상속을 올바르게 사용했다면 파생된 클래스는 부모 클래스의 "전문화"/"구체화" 가 된다
- 그치만 일관성을 가져야 한다
    - 부모 클래스가 가지지 않은 무관한 메서드가 추가되었다면 올바른 상속이라 볼 수 없다
- 그런 경우엔 컴포지션을 활용하자


## 파이썬의 다중상속
- 양날의 검: 유익할수도 있지만 올바르게 구현하지 않으면 문제가 커진다
- 어댑터 패턴이나 믹스인에 활용할 수 있다

### 메서드 결정 순서 (MRO)
- 다이아몬드 문제
    - 파이썬은 메서드 결정 순서를 가지고 있으므로 다이아몬드 문제가 발생하지 않는다

### 믹스인 (mixin)
- "행동"을 캡슐화해놓은 부모 클래스
- 다중 상속한 파생 클래스가 있을 때 의미가 생김

In [22]:
class BaseTokenizer:
    """
    >>> tk = BaseTokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
    >>> list(tk)
    ['28a2320b', 'fd3f', '4627', '9792', 'a2b38e3c46b0']
    """

    def __init__(self, str_token):
        self.str_token = str_token

    def __iter__(self):
        yield from self.str_token.split("-")


class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())


class Tokenizer(UpperIterableMixin, BaseTokenizer):
    """
    >>> tk = Tokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
    >>> list(tk)
    ['28A2320B', 'FD3F', '4627', '9792', 'A2B38E3C46B0']
    """

In [23]:
tokenizer = Tokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
list(tokenizer)

['28A2320B', 'FD3F', '4627', '9792', 'A2B38E3C46B0']

# 함수와 메서드의 인자
## 파이썬의 함수 인자 동작방식
### 인자는 함수에 어떻게 복사되는가
- pass by value

In [24]:
def function(arg):
    arg += " in function"
    print(arg)

immutable = "hello"

function(immutable)

hello in function


In [25]:
mutable = list("hello")
immutable

'hello'

In [26]:
function(mutable)

['h', 'e', 'l', 'l', 'o', ' ', 'i', 'n', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n']


In [27]:
mutable

['h',
 'e',
 'l',
 'l',
 'o',
 ' ',
 'i',
 'n',
 ' ',
 'f',
 'u',
 'n',
 'c',
 't',
 'i',
 'o',
 'n']

- 함수 인자를 함수 안에서 변경하지 말자. 최대한 함수 호출을 통해 발생할 수 있는 부작용을 회피하자

### 가변 인자
- 우리가 아는 그거

### 위치 전용 (positional-only) 인자
